In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import os
import torch
from torch2trt import torch2trt
import warnings
from segment_anything import sam_model_registry, SamPredictor
from segment_anything.utils.onnx import SamOnnxModel

from Module.models.timm_image_encoder import TimmImageEncoder
from mobile_sam.modeling.sam import Sam

C:\Users\danusys\anaconda3\envs\nanosam\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\danusys\anaconda3\envs\nanosam\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
C:\Users\danusys\anaconda3\envs\nanosam\lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
d:\watersegmentation\mobilesam\mobile_sam\modeling\tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_5m_224 in registry with mobile_sam.modeling.tiny_vi

## Model Export

### Common Settomg

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
opset=16

### Encoder

In [3]:
enc_type = 'resnet18'
enc_checkpoint = "./runs/241031_vit-b_to_resnet18_v2/Nanosam_encoder.pth"

In [4]:
## Model load
sam_enc = TimmImageEncoder('resnet18', pretrained=True)
sam_enc.load_state_dict(torch.load(enc_checkpoint)["model"])

sam_enc.to(device)
sam_enc.eval();

In [5]:
input_size=1024
dummy_input = torch.randn(1, 3, input_size, input_size).to(device)

In [6]:
enc_trt = torch2trt(sam_enc, [dummy_input], )

In [7]:
output_enc_path = enc_checkpoint.replace(".pth", ".onnx")

torch.onnx.export(
            sam_enc,
            (dummy_input,),
            output_enc_path,
            input_names=["image"],
            output_names=["image_embeddings"],
            opset_version=opset
        )

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



### Decoder

In [4]:
dec_model_type = 'vit_b'
dec_checkpoint = '../runs/241115_SAM_ViT_b_dataset_V2_ft_v1/best.pth'

sam = sam_model_registry[dec_model_type](checkpoint=dec_checkpoint)

In [5]:
onnx_model = SamOnnxModel(
    model=sam, 
    return_single_mask=True,
    use_stability_score=False,
    return_extra_metrics=False
)

In [17]:
save_dir = os.path.split(dec_checkpoint)[0]

output_name = os.path.split(dec_checkpoint)[1]
output_name = os.path.splitext(output_name)[0]+"_decoder.onnx" 

output_dec_path = os.path.join(save_dir, output_name)
output_dec_path


'../runs/241115_SAM_ViT_b_dataset_V2_ft_v1\\best_decoder.onnx'

In [18]:
dynamic_axes = {
        "point_coords": {1: "num_points"},
        "point_labels": {1: "num_points"},
    }

embed_dim = sam.prompt_encoder.embed_dim
embed_size = sam.prompt_encoder.image_embedding_size
mask_input_size = [4 * x for x in embed_size]
dummy_inputs = {
        "image_embeddings": torch.randn(1, embed_dim, *embed_size, dtype=torch.float),
        "point_coords": torch.randint(low=0, high=1024, size=(1, 5, 2), dtype=torch.float),
        "point_labels": torch.randint(low=0, high=4, size=(1, 5), dtype=torch.float),
        "mask_input": torch.randn(1, 1, *mask_input_size, dtype=torch.float),
        "has_mask_input": torch.tensor([1], dtype=torch.float),
        "orig_im_size": torch.tensor([1500, 2250], dtype=torch.int32),
    }

In [19]:
_ = onnx_model(**dummy_inputs)

In [20]:
output_names = ["masks", "iou_predictions", "low_res_masks"]

In [21]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=torch.jit.TracerWarning)
    warnings.filterwarnings("ignore", category=UserWarning)
    with open(output_dec_path, "wb") as f:
        print(f"Exporting onnx model to {output_dec_path}...")
        torch.onnx.export(
                onnx_model,
                tuple(dummy_inputs.values()),
                f,
                export_params=True,
                verbose=False,
                opset_version=opset,
                do_constant_folding=True,
                input_names=list(dummy_inputs.keys()),
                output_names=output_names,
                dynamic_axes=dynamic_axes,
            )

Exporting onnx model to ../runs/241115_SAM_ViT_b_dataset_V2_ft_v1\best_decoder.onnx...
============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

